# Workbench Quick Reference

Beyond the indexable interface, `PowerWorld` provides convenience
methods that wrap common workflows into single calls. These cover
data retrieval shortcuts, power flow, voltage analysis, matrix
extraction, sensitivity factors, and case control - so you spend
less time assembling field lists and more time on analysis.

```python
from esapp import PowerWorld
from esapp.components import *

pw = PowerWorld("path/to/case.pwb")
```

In [8]:
from esapp import PowerWorld
from esapp.components import *
import numpy as np
import ast

with open('../../../examples/data/case.txt', 'r') as f:
    case_path = ast.literal_eval(f.read().strip())

pw = PowerWorld(case_path)

'open' took: 7.0825 sec


## Data Shortcuts

These methods return DataFrames of common object types with their
most useful fields pre-selected. They're thin wrappers around the
indexable interface - `pw.gens()` is equivalent to
`pw[Gen, ["GenMW", "GenMVR", "GenStatus"]]` but shorter to type.

| Method | Returns |
|---|---|
| `pw.gens()` | Generator MW, Mvar, and status |
| `pw.loads()` | Load MW, Mvar, and status |
| `pw.shunts()` | Switched shunt MW, Mvar, and status |
| `pw.lines()` | All transmission lines |
| `pw.transformers()` | All transformers |
| `pw.areas()` | All areas |
| `pw.zones()` | All zones |

In [9]:
pw.gens().head()

,BusNum,GenID,GenMVR,GenMW,GenStatus
0,2,1,0.80000,2.500000,Closed
1,2,2,0.80000,2.500000,Closed
2,2,3,0.80000,2.500000,Closed
3,2,4,0.80000,2.500000,Closed
4,23,1,0.04408,69.274741,Closed


In [10]:
pw.loads().head()

,BusNum,LoadID,LoadMVR,LoadMW,LoadStatus
0,2,1,0.0,60.699999,Closed
1,3,1,0.0,59.390002,Closed
2,4,1,0.0,22.470000,Closed
3,6,1,0.0,27.460000,Closed
4,7,1,0.0,37.009999,Closed


## Power Flow & Voltage Analysis

`pflow()` solves the AC power flow and returns complex bus voltages.
After solving, all data accessed through the indexable interface
reflects the updated system state. You can specify the solution
method explicitly or use the default (Polar Newton-Raphson).

```python
V = pw.pflow()                       # default: Polar Newton-Raphson
V = pw.pflow(method="POLARNEWT")     # explicit method selection
```

Several methods provide quick access to voltage and power data
without having to assemble field lists manually:

| Method | Returns |
|---|---|
| `pw.voltage()` | Complex bus voltages (per-unit) |
| `pw.voltage(complex=False)` | Magnitude and angle as separate Series |
| `pw.set_voltages(V)` | Set bus voltages from a complex array |
| `pw.mismatch()` | Bus active/reactive power mismatches |
| `pw.netinj()` | Net power injection at each bus |
| `pw.violations(v_min, v_max)` | Buses outside a voltage band |

In [11]:
V = pw.voltage()
np.abs(V).describe()

count    37.000000
mean      0.988868
std       0.008708
min       0.974911
25%       0.980583
50%       0.991217
75%       0.995798
max       1.003416
dtype: float64

## Matrices & Topology

System matrices are returned as scipy sparse arrays by default,
with an option for dense numpy arrays. The bus map provides the
mapping between PowerWorld bus numbers and matrix row/column indices.

| Method | Returns |
|---|---|
| `pw.ybus()` | System admittance matrix (sparse) |
| `pw.ybus(dense=True)` | Dense admittance matrix |
| `pw.jacobian()` | Power flow Jacobian |
| `pw.jacobian(form="P")` | Polar-form Jacobian |
| `pw.jacobian(form="DC")` | DC Jacobian |
| `pw.busmap()` | Bus number to matrix index mapping |
| `pw.buscoords()` | Bus latitude/longitude from substation data |

In [12]:
Y = pw.ybus()
Y.shape

(37, 37)

In [13]:
J = pw.jacobian()
J.shape

(74, 74)

## Case Control & Solver Options

Basic case management methods handle saving, closing, and resetting
the case state. Solver options are exposed as Python descriptors on
`PowerWorld`, letting you read and write simulation settings as
regular attributes rather than calling dedicated getter/setter methods.

| Method | Purpose |
|---|---|
| `pw.save()` | Save the case |
| `pw.save("new.pwb")` | Save to a new file |
| `pw.close()` | Close the current case |
| `pw.flatstart()` | Reset to 1.0 pu / 0 deg |
| `pw.edit_mode()` | Enter edit mode |
| `pw.run_mode()` | Enter run mode |
| `pw.log(msg)` | Add a message to the PowerWorld log |

Solver settings are accessed directly as attributes:

```python
pw.max_iterations = 250       # set max Newton iterations
pw.flat_start = True          # enable flat start
pw.convergence_tol = 1e-5     # set convergence tolerance
pw.do_one_iteration = True    # single iteration mode
pw.dc_mode = True             # DC approximation
```

See the API Reference for the full list of
solver option descriptors.

## Convenience Features

Several methods wrap multi-step workflows into single calls.

**Branch flows and overloads** - `flows()` retrieves MW, MVR, MVA,
and percent loading for every branch. `overloads()` filters to
branches exceeding a threshold.

| Method | Returns |
|---|---|
| `pw.flows()` | Branch MW, MVR, MVA, and % loading |
| `pw.overloads(threshold=100)` | Branches exceeding a loading threshold |

**Sensitivity factors** - `ptdf()` and `lodf()` compute Power
Transfer Distribution Factors and Line Outage Distribution Factors,
respectively. Both call the underlying SAW sensitivity commands and
return the results as a DataFrame.

| Method | Returns |
|---|---|
| `pw.ptdf(seller, buyer)` | Power Transfer Distribution Factors |
| `pw.lodf(branch)` | Line Outage Distribution Factors |

**Snapshot** - a context manager that saves the current case state
on entry and restores it on exit, even if an exception occurs.
Useful for "what-if" analyses where you want to modify the case
temporarily without losing the original state.

```python
with pw.snapshot():
    pw[Gen, "GenMW"] = modified_gen
    pw.pflow()
    v = pw.voltage()
# state automatically restored here
```

**Quick properties** provide case-level counts and the system MVA
base without having to query and count manually. `summary()`
aggregates these into a single dict along with generation/load
totals and the voltage range.

| Property / Method | Returns |
|---|---|
| `pw.n_bus` | Number of buses |
| `pw.n_branch` | Number of branches |
| `pw.n_gen` | Number of generators |
| `pw.sbase` | System MVA base (float) |
| `pw.summary()` | Dict with counts, totals, and voltage range |

## Utility Modules

`PowerWorld` embeds specialized analysis modules as attributes.
These provide domain-specific functionality built on top of the
indexable interface and SAW commands.

| Attribute | Module | Capabilities |
|---|---|---|
| `pw.network` | Network topology | Incidence matrices, Laplacians, bus coordinates |
| `pw.gic` | GIC analysis | G-matrix, E-field Jacobians, storm simulation |

Like the solver options on `PowerWorld`, GIC analysis options are
also exposed as descriptors on `pw.gic`:

```python
pw.gic.pf_include = True          # enable GIC in power flow
pw.gic.calc_mode = 'SnapShot'     # set calculation mode
pw.gic.configure()                # apply sensible defaults
```

See the API Reference for full documentation
of each module.